# SITE demo: Tensor Symbolic Regression quick start

This notebook is a minimal quick-start that shows how to configure and run the SITE tensor symbolic regression example included in the repository.
We keep one small, runnable example that uses reduced population and generations so you can try it locally in a few seconds/minutes. For full experiments, increase the parameters near the bottom of the demo.

The example constructs synthetic data for a 3x3 tensor target T_ij, defines primitive sets and operators, then runs a short evolution to demonstrate the pipeline.

## 1) Imports and helper functions

Run the cell below to import the required modules and define small helper functions used by the demo. If you don't have all dependencies installed, see the README for installation notes (geppy, deap, numpy, scipy, sympy).

In [1]:
# Imports and small helpers for the quick-start demo
import geppy as gep
from deap import creator, base, tools
import numpy as np
import random
import operator
import time

# For reproducibility
s = 0
random.seed(s)
np.random.seed(s)

import SITE
from simplification import p_symbol, linker_add, protected_div_symbol

# Tensor operators used in primitives
def tensor_add(*args):
    args = [arg for arg in args]
    sum_val = 0
    for arg in args:
        sum_val += arg
    return sum_val

def tensor_sub(a, b):
    if isinstance(a, np.ndarray) and isinstance(b, np.ndarray) and a.shape == b.shape:
        return a - b
    else:
        return False

def tensor_inner_product(a, b):
    if isinstance(a, np.ndarray) and isinstance(b, np.ndarray) and a.shape == b.shape:
        result = np.empty_like(a)
        for i in range(a.shape[0]):
            result[i] = np.matmul(a[i], b[i])
        return result
    else:
        return False

def protected_div(x1, x2):
    if isinstance(x2, np.ndarray):
        abs_x2 = np.maximum(x2, -x2)
        if (abs_x2 < 1e-10).any():
            return 1
        return x1 / x2
    else:
        if abs(x2) < 1e-10:
            return 1
        return x1 / x2
    
# a placeholder function
def p_(tensor):
    pass

## 2) Generate synthetic data
Create a small synthetic dataset for the target tensor $T_{ij}$ and the input tensors/scalars.

### Case description
The target equation to be identified in this example is:

$$
T_{ij} = 4A_{ij} + 0.91\, \alpha\, B_{ik} B_{kj} - 3.41\, A_{ik} B_{kj} + 0.5\, \beta\, \delta_{ij}
$$

Here we treat $A_{ij}$ and $B_{ij}$ as input second-order tensors (shape: $[n_\text{points}, 3, 3]$), $\alpha$ and $\beta$ as input scalars (shape: $[n_\text{points}, 1]$), and $\delta_{ij}$ as the identity (Kronecker delta).

Variable distributions used to synthesize the data for the demo:
- $\alpha$: Uniform $(-10, 10)$ scalar per sample
- $\beta$: Uniform $(5, 15)$ scalar per sample
- Nonzero components of $A_{ij}$: sampled from various Uniform ranges (see code) to represent asymmetric tensor entries
- $B_{ij}$: diagonal and a few off-diagonal components sampled from wide ranges (see code) — note $b_{11}$ uses a large magnitude uniform, and $b_{33}$ is small to create scale contrasts.

This synthetic setup ensures the target expression mixes tensor addition, inner products and scalar multipliers with differing magnitudes — a representative small test for the SITE framework.

In [2]:
# Create a compact synthetic dataset (quick-run size)
n_points = 50
alpha = np.random.uniform(-10, 10, n_points).reshape(n_points, 1)
beta = np.random.uniform(5, 15, n_points).reshape(n_points, 1)

A_ij = np.zeros((n_points, 3, 3))
A_ij[:, 0, 1] = np.random.uniform(-10, 10, n_points)
A_ij[:, 0, 2] = np.random.uniform(-2, 0, n_points)
A_ij[:, 1, 0] = np.random.uniform(-10, 10, n_points)
A_ij[:, 1, 2] = np.random.uniform(-10, 10, n_points)
A_ij[:, 2, 1] = np.random.uniform(-10, 10, n_points)

B_ij = np.zeros((n_points, 3, 3))
B_ij[:, 0, 0] = np.random.uniform(-15e2, 5e2, n_points)
B_ij[:, 0, 2] = np.random.uniform(3, 13, n_points)
B_ij[:, 2, 0] = B_ij[:, 0, 2]
B_ij[:, 2, 2] = np.random.uniform(1e-5, 5e-5, n_points)

delta_ij = np.zeros((n_points, 3, 3))
delta_ij[:, 0, 0] = 1
delta_ij[:, 1, 1] = 1
delta_ij[:, 2, 2] = 1

# True generating equation (used to synthesize Y)
T_ij = 4*A_ij + 0.91*alpha[:, None] * tensor_inner_product(B_ij, B_ij) - 3.41*tensor_inner_product(A_ij, B_ij) + 0.5*beta[:, None]*delta_ij
Y = T_ij

# Pack variables to feed into the compile/evaluate functions
dict_of_variables = {'A_ij':A_ij, 'B_ij':B_ij, 'delta_ij':delta_ij, 'alpha':alpha, 'beta':beta}

## 3) Define operators and primitive sets
Define the symbolic/operator maps and primitive sets used by the evolitionary algorithms.

In [3]:
symbolic_function_map = {
    'tensor_add': linker_add,
    'tensor_sub': operator.sub,
    'tensor_inner_product': operator.mul,
    'p_': p_symbol,
    operator.add.__name__: operator.add,
    operator.sub.__name__: operator.sub,
    operator.mul.__name__: operator.mul,
    'protected_div': protected_div_symbol,
}
dict_of_operators = {
    'tensor_add':tensor_add,
    'tensor_sub':tensor_sub,
    'tensor_inner_product':tensor_inner_product,
    operator.add.__name__: operator.add,
    operator.sub.__name__: operator.sub,
    operator.mul.__name__: operator.mul,
    'protected_div':protected_div,
    'linker_add':linker_add
}
dict_of_dimension = {'A_ij':[0], 'B_ij':[0], 'delta_ij':[0], 'alpha':[0], 'beta':[0]}
NUM_UNITS = 1
target_dimension = [0]

# Create primitive sets (host uses tensor ops, plasmid uses scalar ops)
host_pset = gep.PrimitiveSet('Host', input_names=['A_ij','B_ij','delta_ij'])
host_pset.add_function(tensor_add, 2)
host_pset.add_function(tensor_sub, 2)
host_pset.add_function(tensor_inner_product, 2)
host_pset.add_function(p_, 1)

plasmid_pset = gep.PrimitiveSet('Plasmid', input_names=['alpha', 'beta'])
plasmid_pset.add_function(operator.add, 2)
plasmid_pset.add_function(operator.sub, 2)
plasmid_pset.add_function(operator.mul, 2)
plasmid_pset.add_function(protected_div, 2)

## 4) Setup GEP toolbox and populations
Create the DEAP/geppy toolbox, register compile/evaluate functions, and build a small population for quick demo runs.

In [4]:
# Basic GEP/DEAP setup (kept minimal for a quick demo)
creator.create("FitnessMax", base.Fitness, weights=(-1,)) # Minimization of fitness
creator.create("Host_Individual", gep.Chromosome, fitness=creator.FitnessMax, plasmid=[]) # Host individual with plasmids
creator.create("Plasmid_Individual", gep.Chromosome) # Plasmid individual

# Define parameters for the genetic programming
h_host = 5
h_plasmid = 10
n_genes_host = 4
n_genes_plasmid = 1

# Set up the DEAP toolbox with the relevant operators
toolbox = gep.Toolbox()
# Host setup
toolbox.register('host_gene_gen', gep.Gene, pset=host_pset, head_length=h_host)
toolbox.register('host_individual', creator.Host_Individual, gene_gen=toolbox.host_gene_gen, n_genes=n_genes_host, linker=tensor_add)
toolbox.register("host_population", tools.initRepeat, list, toolbox.host_individual)
# Define a seed/tinder individual with a single gene, used to do the seed injection
creator.create("tinder", gep.Chromosome, plasmid=[])
toolbox.register('tinder_individual', creator.tinder, gene_gen=toolbox.host_gene_gen, n_genes=1, linker=tensor_add)
# Plasmid setup
toolbox.register('plasmid_gene_gen', gep.Gene, pset=plasmid_pset, head_length=h_plasmid)
toolbox.register('plasmid_individual', creator.Plasmid_Individual, gene_gen=toolbox.plasmid_gene_gen, n_genes=n_genes_plasmid, linker=linker_add)
toolbox.register("plasmid_population", SITE.plasmid_generate, toolbox.plasmid_individual)

toolbox.register('compile', SITE.my_compile, dict_of_operators=dict_of_operators, symbolic_function_map=symbolic_function_map, dict_of_variables=dict_of_variables, Y=Y)
toolbox.register('dimensional_verification', SITE.dimensional_verification, dict_of_dimension=dict_of_dimension, num_units=NUM_UNITS, target_dimension=target_dimension)
toolbox.register('evaluate', SITE.evaluate, tb=toolbox, dict_of_operators=dict_of_operators, dict_of_variables=dict_of_variables, Y=Y)

# Register simple genetic operators used in the demo
# 0. selection with tournament of size 50
toolbox.register('select', tools.selTournament, tournsize=50)
# 1. general operators for host population
toolbox.register('mut_uniform', SITE.mutate_uniform, host_pset = host_pset, 
                 func = toolbox.plasmid_individual, ind_pb=0.2, pb=1)
toolbox.register('mut_invert', SITE.invert, pb=0.2)
toolbox.register('mut_is_transpose', SITE.is_transpose, pb=0.2)
toolbox.register('mut_ris_transpose', SITE.ris_transpose, pb=0.2)
toolbox.register('mut_gene_transpose', SITE.gene_transpose, pb=0.2)
toolbox.register('cx_1p', SITE.crossover_one_point, pb=0.2)
toolbox.register('cx_2p', SITE.crossover_two_point, pb=0.2)
toolbox.register('cx_gene', SITE.crossover_gene, pb=0.2)
# 2. general operators for plasmid population
toolbox.register('mut_uniform_plasmid', gep.mutate_uniform, pset = plasmid_pset, ind_pb=0.05, pb=1)
toolbox.register('mut_invert_plasmid', gep.invert, pb=0.1)
toolbox.register('mut_is_transpose_plasmid', gep.is_transpose, pb=0.1)
toolbox.register('mut_ris_transpose_plasmid', gep.ris_transpose, pb=0.1)
toolbox.register('mut_gene_transpose_plasmid', gep.gene_transpose, pb=0.1)

# Statistics (minimization of fitness)
stats = tools.Statistics(key=lambda ind: ind.fitness.values[0])
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

## 5) Run the evolution
Run a short evolution with small population and generations to validate the pipeline.

In [5]:
# Small quick-run parameters
n_pop = 400
n_gen = 50
tol = 1e-6
output_type = 'toy_case'

# Initialize populations
host_pop = toolbox.host_population(n=n_pop)
plasmid_pop = toolbox.plasmid_population(host_pop)
for ind_host, ind_plasmid in zip(host_pop, plasmid_pop):
    ind_host.plasmid = ind_plasmid

# Only record the best three individuals ever found in all generations
champs = 3 
hof = tools.HallOfFame(champs)

start_time = time.time()
pop, log = SITE.gep_simple(host_pop, plasmid_pop, toolbox, n_generations=n_gen, n_elites=1, n_alien_inds=5, stats=stats, hall_of_fame=hof, verbose=True, tolerance=tol, GEP_type=output_type)
print(f"\nTotal time: {time.time() - start_time} seconds\n")

gen	nevals	avg     	min     	max     
0  	400   	0.779086	0.380761	0.999997
1  	394   	0.686909	0.260252	1       
2  	394   	0.682429	0.0341516	1       
3  	394   	0.61236 	0.00683602	0.999983
4  	394   	0.568549	0.00683602	0.999983
5  	394   	0.547913	0.00683602	0.999984
6  	394   	0.517814	0.00405743	0.999861
7  	394   	0.53903 	0.00343862	1       
8  	394   	0.544035	0.00343862	0.999953
9  	394   	0.532913	2.86622e-16	0.999991

Total time: 3.658421516418457 seconds

